In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from env import get_connection

# Exercises II

1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [2]:
import pandas as pd
import numpy as np
import os
from env import host, username, password
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

def get_connection(db, user=username, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

def new_zillow_data():
    '''
    This function reads the zillow data from the Codeup db into a df.
    '''
    # Create SQL query.
    sql_query = """
                SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, 
                taxvaluedollarcnt, yearbuilt, taxamount, fips, propertylandusedesc
                FROM properties_2017
                JOIN propertylandusetype USING(propertylandusetypeid)
                WHERE propertylandusedesc = 'Single Family Residential'
                """
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('zillow'))
    
    return df

def get_zillow_data():
    '''
    This function reads in zillow data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('zillow.csv'):
        
        # If csv file exists, read in data from csv file.
        df = pd.read_csv('zillow.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame.
        df = new_zillow_data()
        
        # Write DataFrame to a csv file.
        df.to_csv('zillow.csv')
        
    return df

2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [3]:
# this df variable inputs the zillow.csv file into the jupyter home repository
df = get_zillow_data()

# display the data
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusedesc
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,Single Family Residential
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,Single Family Residential
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,Single Family Residential
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0,Single Family Residential
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0,Single Family Residential


In [4]:
# 2152863 rows and 8 columns
df.shape

(2152863, 8)

In [5]:
# 7 float columns, and 1 object column
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152863 entries, 0 to 2152862
Data columns (total 8 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
 7   propertylandusedesc           object 
dtypes: float64(7), object(1)
memory usage: 147.8+ MB


In [6]:
# summary of the data
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2152852.0,3.287196,0.954754,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2152852.0,2.230688,0.999280,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2144379.0,1862.855178,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2152370.0,461896.237963,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
yearbuilt,2143526.0,1960.949681,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2148421.0,5634.865978,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
fips,2152863.0,6048.377335,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00


In [7]:
# null values
df.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
propertylandusedesc                0
dtype: int64

In [8]:
# list of columns
df.columns

Index(['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'taxvaluedollarcnt', 'yearbuilt', 'taxamount', 'fips',
       'propertylandusedesc'],
      dtype='object')

In [9]:
# drop the nulls
df = df.dropna()
df.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
propertylandusedesc             0
dtype: int64

In [10]:
# drop column using .drop(columns=column_name)
df = df.drop(columns='propertylandusedesc')

In [11]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0


In [12]:
df.yearbuilt = df.yearbuilt.astype(int)
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973,255.17,6037.0


In [13]:
df.fips = df.fips.astype(int)

In [14]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005,6941.39,6037
6,3.0,4.0,1620.0,847770.0,2011,10244.94,6037
7,3.0,2.0,2077.0,646760.0,1926,7924.68,6037
11,0.0,0.0,1200.0,5328.0,1972,91.60,6037
14,0.0,0.0,171.0,6920.0,1973,255.17,6037


In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# ------------------- BASIC SPLIT FUNCTION  -------------------

def my_train_test_split(df, target):
    train, test = train_test_split(df, test_size=.2, random_state=123, stratify=df[target])
    train, validate = train_test_split(train, test_size=.25, random_state=123, stratify=train[target])
    return train, validate, test
    

# ------------------- ZILLOW DATA -------------------

def clean_zillow(df):

    '''Prepares acquired zillow data for exploration'''
    
    # drop column using .drop(columns=column_name)
    df = df.drop(columns= 'propertylandusedesc')
    
    # drop the nulls
    df = df.dropna()
    
    # convert floats to int
    df.yearbuilt = df.yearbuilt.astype(int)
    df.fips = df.fips.astype(int)
    df.calculatedfinishedsquarefeet = df.calculatedfinishedsquarefeet.astype(int)
    df.taxvaluedollarcnt = df.taxvaluedollarcnt.astype(int)
    
    # rename columns
    df = df.rename(columns={'calculatedfinishedsquarefeet':'square_footage'})
    df = df.rename(columns={'taxvaluedollarcnt':'tax_value'})
    
    return df


def split_zillow_data(df):
    '''
    take in a DataFrame and return train, validate, and test DataFrames; stratify on tax_value.
    return train, validate, test DataFrames.
    '''
    
    # splits df into train_validate and test using train_test_split() stratifying on tax_value to get an even mix of each tax amount
    train_validate, test = train_test_split(df, test_size=.2, random_state=123, stratify=df.tax_value)
    
    # splits train_validate into train and validate using train_test_split() stratifying on species to get an even mix of each species
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=123, 
                                       stratify=train_validate.tax_value)
    return train, validate, test


def wrangle_zillow(df):
    '''Prepares acquired zillow data for exploration'''
    
    # drop column using .drop(columns=column_name)
    df = df.drop(columns= 'propertylandusedesc')
    
    # drop the nulls
    df = df.dropna()
    
    # convert floats to int
    df.yearbuilt = df.yearbuilt.astype(int)
    df.fips = df.fips.astype(int)
    df.calculatedfinishedsquarefeet = df.calculatedfinishedsquarefeet.astype(int)
    df.taxvaluedollarcnt = df.taxvaluedollarcnt.astype(int)
    
    # rename columns
    df = df.rename(columns={'calculatedfinishedsquarefeet':'square_footage'})
    df = df.rename(columns={'taxvaluedollarcnt':'tax_value'})
    
    # split data into train/validate/test using split_data function
    train, validate, test = split_zillow_data(df)
    
    return train, validate, test

3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow.

In [31]:
import wrangle

In [23]:
df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusedesc
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,Single Family Residential
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,Single Family Residential
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,Single Family Residential
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0,Single Family Residential
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0,Single Family Residential
...,...,...,...,...,...,...,...,...
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0,Single Family Residential
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0,Single Family Residential
2152860,0.0,0.0,NaN,208057.0,NaN,5783.88,6059.0,Single Family Residential
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0,Single Family Residential
